In [72]:
#No me cambien las cosas perros
import pandas as pd
from sodapy import Socrata
import seaborn as sns

#aqui sacamos las tablas de datos de hospitales y desfibriladores 
client = Socrata("analisi.transparenciacatalunya.cat", None)

desfibriladors = client.get("wpud-ukyg")
hospitales = client.get("8gmd-gz7i")

df_desfibriladors = pd.DataFrame.from_records(desfibriladors) 
df_hospitales = pd.DataFrame.from_records(hospitales) 
df_socioeconomic = pd.read_csv("socio_eco.csv")

In [88]:

#CÓDIGO QUE LIMPIA Y PROCESA LOS DATOS SOCIOECONÓMICOS, DEJA UNA COLUMNA DE PUNTUACIÓN SOCIOECONÓMICA QUE USARÉ PARA EL MAPA
import pandas as pd
from sodapy import Socrata
import seaborn as sns

df_socioeconomic = pd.read_csv("socio_eco.csv")
df_socioeconomic = df_socioeconomic[['any', 'municipi', 'concepte', 'valor']]
# Limpiar valores vacíos o '..'
df_socioeconomic['valor'] = pd.to_numeric(df_socioeconomic['valor'], errors='coerce')

# Pivotar
df_socioeconomic = df_socioeconomic.pivot_table(
    index=['any', 'municipi'],
    columns='concepte',
    values='valor',
    aggfunc='first'
).reset_index()

# ahora sustituyo los valores NaN por la media de la columna
for column in df_socioeconomic.columns:
    if df_socioeconomic[column].dtype in ['float64', 'int64']:
        mean_value = df_socioeconomic[column].mean()
        df_socioeconomic[column].fillna(mean_value, inplace=True)

df_socioeconomic['població desocupada(%)'] = 100-df_socioeconomic['població ocupada (%)']
df_socioeconomic.drop(columns=['població ocupada (%)'], inplace=True)

#normalizo cada columna numérica sin sklearn
numeric_columns = df_socioeconomic.select_dtypes(include=['float64', 'int64']).columns
for column in numeric_columns:
    min_value = df_socioeconomic[column].min()
    max_value = df_socioeconomic[column].max()
    df_socioeconomic[column] = (df_socioeconomic[column] - min_value) / (max_value - min_value)
df_socioeconomic.drop(columns=['any'], inplace=True)
df_socioeconomic.drop(columns=['estrangers de països de renda baixa o mitjana (%)'], inplace=True)
#ahora creo una columna de puntuación de pobreza socioeconómica, que haga la media sumando o restando algunas columnas
df_socioeconomic['puntuació pobreza socioeconòmica'] = (
    df_socioeconomic['població amb estudis baixos (%)'] * (1/6) +
    df_socioeconomic['població jove sense estudis postobligatoris (%)'] * (1/6) -
    df_socioeconomic['renda mitjana per persona (€)'] * (1/6) +
    df_socioeconomic['treballadors de baixa qualificació (%)'] * (1/6) +
    df_socioeconomic['població desocupada(%)'] * (1/6)
)



C:\Users\German\AppData\Local\Temp\ipykernel_27940\375760513.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_socioeconomic[column].fillna(mean_value, inplace=True)
